In [ ]:
# |default_exp mongo

# DB variables

In [ ]:
# |export

from dotenv import load_dotenv, find_dotenv
from helper import load_env
import os


def get_DB_VARS():
    load_env()
    DB_HOST = os.getenv("DB_HOST")
    DB_NAME = os.getenv("DB_NAME")
    DB_LOGS = os.getenv("DB_LOGS")
    return DB_HOST, DB_NAME, DB_LOGS

# logs

In [ ]:
# |export

from connection import get_collection
from datetime import datetime
import pymongo


def mongolog(operation: str, entite: str, desc: str):
    DB_HOST, DB_NAME, DB_LOGS = get_DB_VARS()
    if DB_LOGS in ["true", "True"]:
        coll_logs = get_collection(DB_HOST, DB_NAME, "logs")
        coll_logs.insert_one(
            {
                "operation": operation,
                "entite": entite,
                "desc": desc,
                "date": datetime.now(),
            }
        )


def print_logs(n: int = 10):
    """
    Print the last n logs
    """
    DB_HOST, DB_NAME, DB_LOGS = get_DB_VARS()
    coll_logs = get_collection(DB_HOST, DB_NAME, "logs")
    for i, log in enumerate(coll_logs.find().sort("date", pymongo.DESCENDING)):
        if i == n:
            break
        print(log)

In [ ]:
print_logs(5)

{'_id': ObjectId('67793863c7f167e996f82efb'), 'operation': 'delete', 'entite': 'auteur', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 4, 14, 32, 19, 746000)}
{'_id': ObjectId('67793863c7f167e996f82ef8'), 'operation': 'insert', 'entite': 'auteur', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 4, 14, 32, 19, 716000)}
{'_id': ObjectId('6779369a928f2e89a582e72b'), 'operation': 'delete', 'entite': 'auteur', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 4, 14, 24, 42, 692000)}
{'_id': ObjectId('67793698928f2e89a582e728'), 'operation': 'insert', 'entite': 'auteur', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 4, 14, 24, 40, 307000)}
{'_id': ObjectId('67793468928f2e89a582e725'), 'operation': 'delete', 'entite': 'auteur', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 4, 14, 15, 20, 184000)}


# Auteur

In [ ]:
# |export

from bson import ObjectId
from connection import get_collection


class Auteur:
    def __init__(self, nom: str):
        """
        Auteur is a class that represents an author in the database auteurs.
        :param nom: The name of the author.
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        self.collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name="auteurs"
        )
        self.nom = nom

    def exists(self) -> bool:
        """
        Check if the author exists in the database.
        :return: True if the author exists, False otherwise.
        """

        return self.collection.find_one({"nom": self.nom}) is not None

    def keep(self):
        """
        Keep the author in the database.
        """

        if not self.exists():
            mongolog("insert", "auteur", self.nom)
            self.collection.insert_one({"nom": self.nom})
        else:
            mongolog("update", "auteur", self.nom)

    def remove(self):
        """
        Remove the author in the database.
        """
        # remove from database

        self.collection.delete_one({"nom": self.nom})
        mongolog("delete", "auteur", self.nom)

    def get_oid(self) -> ObjectId:
        """
        Get the object id of the author.
        :return: The object id of the author. (bson.ObjectId)
        None if does not exist.
        """

        document = self.collection.find_one({"nom": self.nom})
        if document:
            return document["_id"]
        else:
            return None

In [ ]:
acteur1 = Auteur("Victor Hugo")
acteur1.exists()

False

In [ ]:
acteur1.keep()
acteur1.exists()

True

In [ ]:
acteur1.get_oid()

ObjectId('67793863c7f167e996f82ef9')

In [ ]:
acteur1.remove()
acteur1.exists()

False

In [ ]:
acteur1.get_oid()

# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("mongo helper.ipynb", ".")